In [10]:
import feedparser
from bs4 import BeautifulSoup

def fetch_medium_articles(user_name):
    rss_url = f"https://medium.com/feed/@{user_name}"
    feed = feedparser.parse(rss_url)

    articles = []
    for entry in feed.entries:
        soup = BeautifulSoup(entry.summary, "html.parser")
        
        # Fetch the cover image
        img = soup.find("img")
        img_url = img["src"] if img else None

        # Fetch the subtitle (the first paragraph in the content can serve as a subtitle)
        subtitle = soup.p.text if soup.p else None

        articles.append({
            "title": entry.title,
            "subtitle": subtitle,
            "url": entry.link,
            "cover_image": img_url,
        })

    return articles

def insert_articles_into_html(articles, html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    medium_section = soup.find('section', class_='medium-articles')
    if not medium_section:
        print("Couldn't find the medium-articles section!")
        return

    # Remove all existing publications in the medium-articles section
    for pub in medium_section.find_all('div', class_='publication'):
        pub.decompose()

    for article in articles:
        # You can modify this to change the layout
        article_html = f'''
        <div class="publication">
            <img src="{article['cover_image']}" alt="{article['title']}">
            <div class="publication-content">
                <p><strong>{article['title']}</strong></p>
                <p>{article['subtitle']}</p>
                <a href="{article['url']}" target="_blank">Read the article</a>
            </div>
        </div>
        '''
        medium_section.append(BeautifulSoup(article_html, 'html.parser'))

    with open(html_path, 'w', encoding='utf-8') as file:
        file.write(str(soup))

# Example usage:
user_name = "aspras01"
articles = fetch_medium_articles(user_name)
insert_articles_into_html(articles, 'publications.html')